In [39]:
import io
import json
import os
import sys
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
from scipy import stats
import datetime
import time
import statistics as st
from tqdm import tqdm
import requests_cache
import concurrent

In [2]:
headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}


requests_cache.install_cache('turf_cache')

In [3]:
def get_programme(debut, fin):
    programme = []
    url = f"https://www.letrot.com/fr/courses/calendrier-resultats?publish_up={debut}&publish_down={fin}"
    r = requests.get(url, headers=headers)
    soup = bs(r.text, "html.parser")
    reunion_raw = soup.find_all("a", {"class": "racesHippodrome"})
    
    current_date_reunion = "0"
    current_programme = {}
    
    
    for i in tqdm(range(len(reunion_raw))):
        reunion = reunion_raw[i]
        date = reunion.get("href").split("/")[-2]
        hippodrome = reunion.text[2:].strip().replace(" (A ", " ").replace(")", "")
        
        
        date_pmu = "".join(date.split("-")[::-1])
        
#         if current_date_reunion != date_pmu:
#             current_date_reunion = date_pmu
#             current_programme = requests.get(f"https://online.turfinfo.api.pmu.fr/rest/client/65/programme/{date_pmu}/", headers=headers).json()

        numReunion = 0
# #         print(hippodrome)
#         for reunion_pmu in current_programme["programme"]["reunions"]:
#             if hippodrome in reunion_pmu["hippodrome"]["libelleCourt"]:
#                 numReunion = reunion_pmu["numOfficiel"]
#                 break
        
#         if numReunion == 0:
#             continue
        
        course = {"date": date, "idHippo": reunion.get("href").split("/")[-1], "Hippodrome": hippodrome, "lien": reunion.get("href")}
        course["numReunion"] = numReunion
        programme.append(course)
    
    return pd.DataFrame(programme)

In [6]:
reu = get_programme("01-11-2022", "03-11-2022")
reu.head()

100%|██████████| 11/11 [00:00<?, ?it/s]


,date,idHippo,Hippodrome,lien,numReunion
0,2022-11-01,5305,LAVAL,/stats/courses/programme/2022-11-01/5305,0
1,2022-11-01,5901,LE CROISE-LAROCHE,/stats/courses/programme/2022-11-01/5901,0
2,2022-11-01,8201,BEAUMONT DE LOMAGNE,/stats/courses/programme/2022-11-01/8201,0
3,2022-11-01,5006,CHERBOURG,/stats/courses/programme/2022-11-01/5006,0
4,2022-11-01,1407,LISIEUX,/stats/courses/programme/2022-11-01/1407,0


In [7]:
reu

,date,idHippo,Hippodrome,lien,numReunion
0,2022-11-01,5305,LAVAL,/stats/courses/programme/2022-11-01/5305,0
1,2022-11-01,5901,LE CROISE-LAROCHE,/stats/courses/programme/2022-11-01/5901,0
2,2022-11-01,8201,BEAUMONT DE LOMAGNE,/stats/courses/programme/2022-11-01/8201,0
3,2022-11-01,5006,CHERBOURG,/stats/courses/programme/2022-11-01/5006,0
4,2022-11-01,1407,LISIEUX,/stats/courses/programme/2022-11-01/1407,0
5,2022-11-01,6902,LYON LA SOIE,/stats/courses/programme/2022-11-01/6902,0
6,2022-11-01,3001,NIMES,/stats/courses/programme/2022-11-01/3001,0
7,2022-11-01,4417,PONTCHATEAU,/stats/courses/programme/2022-11-01/4417,0
8,2022-11-02,1302,MARSEILLE VIVAUX,/stats/courses/programme/2022-11-02/1302,0
9,2022-11-02,3103,TOULOUSE,/stats/courses/programme/2022-11-02/3103,0


In [49]:
r = requests.get("https://www.letrot.com/stats/courses/programme/2018-11-01/5305/json", headers=headers)
r.json()

{'toReload': False,
 'nomHippodrome': 'LAVAL',
 'course': [{'heureCourse': '12:20',
   'discipline': 'Attelé',
   'numCourse': 1,
   'prix': 'PRIX DU CREDIT AGRICOLE',
   'allocation': '22 000',
   'distance': '2 875',
   'categorie': 'Course E',
   'typePiste': '',
   'conditionsEngagement': "Pour 6 ans, n'ayant pas gagné 135.000. - Recul de 25 m. à 76.000.<br />",
   'hasTracking': False,
   'hasVideoHeat': False,
   'statut': 16,
   'classement': '16 - 15 - 10 - 5 - 6',
   'linkPrix': 'https://www.letrot.com/stats/fiche-course/2018-11-01/5305/1/resultats/arrivee-definitive',
   'replay': '<a href="https://www.letrot.com/fr/replay-courses/2018-11-01/5305/1" class="btn" target="_blank"><svg width="8px" height="12px" viewBox="0 0 8 12" version="1.1" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><g id="Page-1" stroke="none" stroke-width="1" fill="none" fill-rule="evenodd"><polygon id="triangle" fill="#ffffff" transform="translate(4.000000, 6.000000) rotat

In [8]:
def courses(reunions):
    courses_list = []
    
    participants_list = []
    
#     participants_df = pd.DataFrame(participants_list)
#     participants_df.to_csv("participants.csv")
    
    for i, row in reunions.iterrows():
        url = f"https://www.letrot.com/{row['lien']}/json"
        date_pmu = "".join(row["date"].split("-")[::-1])    
        r = requests.get(url, headers=headers)
        courses = r.json()
        for c in courses["course"]:
            if c["discipline"] == "Attelé":
                course_id = row["date"].replace("-", "") + str(row["idHippo"]) + str(c["numCourse"])
#                 participants_pmu = requests.get(f"https://online.turfinfo.api.pmu.fr/rest/client/65/programme/{date_pmu}/R{row['numReunion']}/C{c['numCourse']}/participants", headers=headers)
#                 try:
#                     participants = participants_pmu.json()["participants"]
#                 except:
#                     continue
#                 participants_with_id = [dict(item, **{"id": course_id}) for item in participants]    
#                 participants_list.extend(participants_with_id)
                
                courses_list.append({"date": row["date"], "id": course_id, "hippodrome": courses["nomHippodrome"], "idHippo": row["idHippo"],**c})
    return pd.DataFrame(courses_list)

In [9]:
courses_df = courses(reu)
courses_df.head()

,date,id,hippodrome,idHippo,heureCourse,discipline,numCourse,prix,allocation,distance,categorie,typePiste,conditionsEngagement,hasTracking,hasVideoHeat,statut,classement,linkPrix,replay
0,2022-11-01,2022110153051,LAVAL,5305,13:58,Attelé,1,PRIX PARIS-TURF,20 000,2 850,Course F,,Course Nationale<br />LeTROT Open des Régions ...,False,False,16,1 - 4 - 2 - 7 - 6,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
1,2022-11-01,2022110153052,LAVAL,5305,14:33,Attelé,2,PRIX MOET & CHANDON,20 000,2 850,Course F,,Course Nationale<br />LeTROT Open des Régions ...,False,True,16,10 - 1 - 14 - 15 - 11,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
2,2022-11-01,2022110153053,LAVAL,5305,15:15,Attelé,3,GRAND PRIX DE LA VILLE DE LAVAL,70 000,2 850,Course A,,"Course Européenne<br />Pour 5 à 10 ans inclus,...",False,True,16,7 - 1 - 6 - 11 - 8,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
3,2022-11-01,2022110153054,LAVAL,5305,15:52,Attelé,4,PRIX ANDRE-LOUIS DREUX,30 000,2 850,Course C,,Course Nationale<br />LeTROT Open des Régions ...,False,True,16,15 - 12 - 6 - 16 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."
4,2022-11-01,2022110153055,LAVAL,5305,16:26,Attelé,5,PRIX CREDIT AGRICOLE ANJOU MAINE,27 000,2 850,Course E,,"Pour 6 ans, n'ayant pas gagné 135.000. - Recul...",False,True,16,5 - 4 - 8 - 14 - 2,https://www.letrot.com/stats/fiche-course/2022...,"<a href=""https://www.letrot.com/fr/replay-cour..."


In [38]:
def info_tableau_partant(courseId, date, idHippo, numCourse, classement):
    chevaux = []
    url = f"https://www.letrot.com/stats/fiche-course/{date}/{idHippo}/{numCourse}/partants/tableau"
    r = requests.get(url, headers=headers)
    soup = bs(r.text, "html.parser")
    headers_table = soup.find("table", {"id": "result_table"}).find("thead").find("tr").find_all("th")
    table = soup.find("table", {"id": "result_table"}).find("tbody")
    rows = table.find_all("tr")
    
    url_arrivee = f"https://www.letrot.com/stats/fiche-course/{date}/{idHippo}/{numCourse}/resultats/arrivee-definitive"
    r_arrivee = requests.get(url_arrivee, headers=headers)
    soup_arrivee = bs(r_arrivee.text, "html.parser")
    table_arrivee = soup_arrivee.find("table", {"id": "result_table"}).find("tbody")
    rows_arrivee = table_arrivee.find_all("tr")
    
    classement = {row.select("td")[1].text : row.select("td")[0].find("span", {"class": "bold"}).text for row in rows_arrivee}
    
#     print(classement)
    
    for i,row in enumerate(rows):
            num = row.select("td")[0].find("span", {"class": "bold"}).text
            col = row.select("td")
            cheval = {}
            cheval["num"] = num
            cheval["nom"] = col[1].text
            
            if num == "NP":
                continue
            cheval["classement"] = classement[num]
            cheval["id"] = courseId
            cheval["date"] = date
            cheval["url"] = col[1].find("a").get("href")
            
            #cheval.update(info_cheval(cheval["url"], date))

            cheval["fer"] = int(col[3].text) if col[3].text else 0
            cheval["firstTimeFer"] = True if col[3].find("div", {"class", "fer-first-time"}) else False
            cheval["sex"] = 0 if col[4].text == "M" else 1
            cheval["age"] = int(col[5].text)
            cheval["dist"] = int(col[6].text.replace(" ", "").replace("\n", ""))
            cheval["driver"] = col[7].find("a").get("href")
            cheval["trainer"] = col[8].find("a").get("href")

            if "Avis" in headers_table[9].text:
                cheval["avisTrainer"] = int(col[9].get("data-order"))
                avis = col.pop(9)
                col.insert(-1, avis)
            else:
                cheval["avisTrainer"] =2

            cheval["music"] = list(filter(lambda x: "a" in x, col[9].text.replace("D", "0").replace("Ret", "0").replace("T", "0").split()))
            cheval["music"] = list(map(lambda x: x[0], cheval["music"]))

            cheval["music"] = list(filter(lambda x: x.isnumeric(), cheval["music"]))

            cheval["music"] = list(map(int, cheval["music"]))
            cheval["nbArrivé"] = len(cheval["music"]) - cheval["music"].count("0")
            cheval["lastPerf"] = cheval["music"][0] if cheval["nbArrivé"] else 0

            arriveOnly = list(filter(None, cheval["music"]))
            if len(arriveOnly) > 0:
                try:
                    cheval["meanPerf"] = np.mean(arriveOnly)
                    cheval["medianPerf"] = np.median(arriveOnly)
                    cheval["modePerf"] = st.mode(cheval["music"])
                except:
                    cheval["meanPerf"] = 0
                    cheval["medianPerf"] = 0
                    cheval["modePerf"] = 0
            else:
                cheval["meanPerf"] = 0
                cheval["medianPerf"] = 0
                cheval["modePerf"] = 0

            try:
                cheval["recordAbs"] = list(map(int, col[10].text.replace(col[10].span.text, "").replace("\'", '"').split('"')))
                cheval["recordAbs"] = cheval["recordAbs"][0] * 10 * 60 + cheval["recordAbs"][1] * 10 + cheval["recordAbs"][2]
            except:
                cheval["recordAbs"] = None

            cheval["gain"] = int(col[11].find("div", class_="gains").text.replace(" ", "")[:-1])

            chevaux.append(cheval)
    return pd.DataFrame(chevaux)

In [11]:
def info_couple(courseId, date, idHippo, numCourse):
    couple_info = []
    date_splitted_arrive = "%2F".join(date.split("-")[::-1])
    date_splitted_depart = date.split("-")[::-1]
    date_splitted_depart[2] = str(int(date_splitted_depart[2]) - 1)
    date_splitted_depart = "%2F".join(date_splitted_depart)
    
    url = f"https://www.letrot.com/stats/fiche-course/{date}/{idHippo}/{numCourse}/partants/couples/paginate?datepicker_du={date_splitted_depart}&datepicker_au={date_splitted_arrive}"
    r = requests.get(url, headers=headers)
    dic_json = r.json()
    data = dic_json["data"]

    data_sorted = sorted(data, key=lambda x: x["numero"])
    for couple in data_sorted:
        cheval = {}
        
        cheval["nbCourseCouple"] = bs(couple["nbre_courses"], "html.parser").find("div").text
        cheval["nbVictoiresCouple"] = bs(couple["nbre_victoires"], "html.parser").find("div").text
        cheval["nb2emeCouple"] = bs(couple["nbre_2eme"], "html.parser").find("div").text
        cheval["nb3emeCouple"] = bs(couple["nbre_3eme"], "html.parser").find("div").text
        cheval["txReussiteCouple"] = couple["taux_reussite_sort"]
        cheval["nonPartant"] = couple["nonPartant"]
        couple_info.append(cheval)
    return pd.DataFrame(couple_info)

In [19]:
def info_cheval(url, date):
    r = requests.get(url + "-paginate-2", headers=headers)
    date_debut = datetime.date.fromisoformat(date)
    jsoned = r.json()["data"]

    info_dict = {}

    for c in jsoned:
        c["dateCourse"] = datetime.date.fromisoformat(c["dateCourseRaw"])
    
    filtered = list(filter(lambda x: x["dateCourse"] < date_debut and x["specialite"] == "A", jsoned))
    
    info_dict["tpsLastRace"] = (date_debut - filtered[0]["dateCourse"]).days
    
    info_dict["last_race_dist"] = int(filtered[0]["distance"].replace(" ", ""))

    return info_dict

In [57]:
def partants(course):
    partants = None
    for i, course in course.iterrows():
        url = f"https://www.letrot.com/stats/fiche-course/{course['date']}/{course['idHippo']}/{course['numCourse']}"
        url_tableau_partant = url + "/partants/tableau"
        url_cheval = url + "/partants/chevaux"
        
        dict_tableau_partant = info_tableau_partant(course['id'],course['date'], course['idHippo'],course['numCourse'], course["classement"])
        dict_couple = info_couple(course['id'],course['date'], course['idHippo'],course['numCourse'])
        
#         combined = dict_tableau_partant.join(dict_couple)
        
        combined = pd.concat([dict_tableau_partant, dict_couple], axis=1)
        
#         combined = combined.join(participants)
        
        if not isinstance(partants, pd.DataFrame):
            partants = combined
        else:
            partants = pd.concat([partants, combined])
        
    return pd.DataFrame(partants)

In [163]:
url = "https://www.letrot.com/stats/fiche-cheval/grizman/ZWR8ZwoIAwQF/courses/dernieres-performances-paginate-2"
r = requests.get(url, headers=headers)
rjson = r.json()["data"]
rjson[0]["dateCourse"]
table.find("thead").find("tr").find_all("th")[9]
list(filter(lambda x: "a" in x, table.find("tbody").find_all("tr")[9].select("td")[9].text.replace("D", "0").split()))

AttributeError: 'str' object has no attribute 'span'

In [58]:
res = partants(courses_df)

In [59]:
res

,num,nom,classement,id,date,url,fer,firstTimeFer,sex,age,...,medianPerf,modePerf,recordAbs,gain,nbCourseCouple,nbVictoiresCouple,nb2emeCouple,nb3emeCouple,txReussiteCouple,nonPartant
0,1,INKY PERRINE,1,2022110153051,2022-11-01,https://www.letrot.com/stats/fiche-cheval/inky...,2.0,False,1.0,4.0,...,4.0,0.0,752.0,17620.0,2,1,0,0,50,False
1,2,IKA JOSSELYN,3,2022110153051,2022-11-01,https://www.letrot.com/stats/fiche-cheval/ika-...,2.0,False,1.0,4.0,...,6.0,0.0,755.0,18120.0,1,0,0,1,100,False
2,3,IDOLE DES SPORTS,0,2022110153051,2022-11-01,https://www.letrot.com/stats/fiche-cheval/idol...,0.0,False,1.0,4.0,...,4.0,9.0,774.0,18850.0,5,0,0,1,20,False
3,4,IDOLE HAUFOR,2,2022110153051,2022-11-01,https://www.letrot.com/stats/fiche-cheval/idol...,3.0,False,1.0,4.0,...,4.0,3.0,753.0,20350.0,1,0,1,0,100,False
4,5,IDEE SECRETE,7,2022110153051,2022-11-01,https://www.letrot.com/stats/fiche-cheval/idee...,2.0,False,1.0,4.0,...,6.0,0.0,758.0,20410.0,6,0,0,1,16,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,12,HELENCIO,8,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hele...,3.0,False,0.0,5.0,...,3.5,3.0,712.0,105720.0,5,0,0,2,40,False
12,13,HIATUS,6,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hiat...,3.0,False,1.0,5.0,...,4.0,0.0,720.0,105830.0,2,0,0,0,0,False
13,14,HARRY CARISAIE,5,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/harr...,3.0,False,1.0,5.0,...,2.0,1.0,733.0,106320.0,4,2,0,0,50,False
14,15,HONNEUR DU CEBE,0,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/honn...,0.0,False,1.0,5.0,...,5.0,0.0,722.0,107280.0,1,0,0,0,0,False


In [23]:
res.to_csv("test.csv")

In [106]:
list(res)

['classement',
 'nom',
 'id',
 'date',
 'url',
 'fer',
 'firstTimeFer',
 'sex',
 'age',
 'dist',
 'driver',
 'trainer',
 'avisTrainer',
 'music',
 'nbArrivé',
 'lastPerf',
 'meanPerf',
 'medianPerf',
 'modePerf',
 'recordAbs',
 'gain',
 'nbCourseCouple',
 'nbVictoiresCouple',
 'nb2emeCouple',
 'nb3emeCouple',
 'txReussiteCouple',
 'nonPartant',
 'oeilleres']

In [159]:
datetime.date.fromisoformat("2020-10-30") < datetime.date(2021,11,2)

True

In [170]:
(datetime.date.fromisoformat("2020-10-30") - datetime.date(2021,11,2)).days

-368

In [194]:
res.to_csv("test2.csv")

# Prediction

In [104]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from statsmodels.discrete.conditional_models import ConditionalMNLogit,ConditionalLogit
import pylogit as pl

from collections import OrderedDict

In [107]:
df = pd.read_csv("test.csv")
df = df.dropna()
df["win"] = df["classement"] == 1
df["availability"] = 1 - df["nonPartant"]
df.head()

,iid,classement,nom,id,date,url,fer,firstTimeFer,sex,age,...,recordAbs,gain,nbCourseCouple,nbVictoiresCouple,nb2emeCouple,nb3emeCouple,txReussiteCouple,nonPartant,win,availability
0,0,0,GOLIATH DU CAUX,2022103080021,2022-10-30,https://www.letrot.com/stats/fiche-cheval/goli...,3,False,1,6,...,719.0,78820,7,4,1,0,71,False,False,1
1,1,0,GAMIN DU LERRE,2022103080021,2022-10-30,https://www.letrot.com/stats/fiche-cheval/gami...,3,False,1,6,...,728.0,79970,1,0,0,0,0,False,False,1
2,2,1,GOUTTE DE PLUIE,2022103080021,2022-10-30,https://www.letrot.com/stats/fiche-cheval/gout...,3,False,1,6,...,737.0,77690,5,2,1,1,80,False,True,1
3,3,0,GUELIZ,2022103080021,2022-10-30,https://www.letrot.com/stats/fiche-cheval/guel...,6,False,1,6,...,725.0,78110,3,0,0,0,0,False,False,1
4,4,0,GAYA DU VARLET,2022103080021,2022-10-30,https://www.letrot.com/stats/fiche-cheval/gaya...,0,False,1,6,...,729.0,79860,4,0,0,0,0,False,False,1


In [95]:
ind_variable = ['fer','firstTimeFer','sex','age','avisTrainer','nbArrivé','lastPerf','meanPerf',
 'medianPerf',
 'modePerf',
 'recordAbs',
 'gain',
 'nbCourseCouple',
 'nbVictoiresCouple',
 'nb2emeCouple',
 'nb3emeCouple',
 'txReussiteCouple',
 'nonPartant']

long_df = pl.convert_wide_to_long(df,ind_variable, {}, {}, "id", "win")

C:\Users\raves\anaconda3\lib\site-packages\pylogit\choice_tools.py:454: UserWarning: Note, there are 29 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 20 variables.
  warnings.warn(msg.format(num_dataframe_vars, data_title) +


ValueError: The values in wide_data[obs_id_col] are not unique, but they need to be.

In [105]:
basic_specification = OrderedDict()
basic_names = OrderedDict()

In [106]:
pl_model = pl.create_choice_model(data=df,alt_id_col="id", obs_id_col="iid", choice_col="win", specification=basic_specification,model_type="MNL")

ValueError: need at least one array to concatenate

In [90]:
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
classement,0,0,1,0,0,0,0,0,3,2
nom,GOLIATH DU CAUX,GAMIN DU LERRE,GOUTTE DE PLUIE,GUELIZ,GAYA DU VARLET,GOWEN CHALEONNAIS,GALA D'ARTISTE,GOLD DU CHATELET,GANADOR DE LAUNI,GUNICA DE CAHOT
id,2022103080021,2022103080021,2022103080021,2022103080021,2022103080021,2022103080021,2022103080021,2022103080021,2022103080021,2022103080021
date,2022-10-30,2022-10-30,2022-10-30,2022-10-30,2022-10-30,2022-10-30,2022-10-30,2022-10-30,2022-10-30,2022-10-30
url,https://www.letrot.com/stats/fiche-cheval/goli...,https://www.letrot.com/stats/fiche-cheval/gami...,https://www.letrot.com/stats/fiche-cheval/gout...,https://www.letrot.com/stats/fiche-cheval/guel...,https://www.letrot.com/stats/fiche-cheval/gaya...,https://www.letrot.com/stats/fiche-cheval/gowe...,https://www.letrot.com/stats/fiche-cheval/gala...,https://www.letrot.com/stats/fiche-cheval/gold...,https://www.letrot.com/stats/fiche-cheval/gana...,https://www.letrot.com/stats/fiche-cheval/guni...
fer,3,3,3,6,0,1,1,3,0,3
firstTimeFer,False,False,False,False,False,False,False,False,False,False
sex,1,1,1,1,1,1,1,1,1,1
age,6,6,6,6,6,6,6,6,6,6
dist,2400,2400,2400,2400,2400,2400,2400,2400,2400,2400


In [129]:
X = df[['fer','firstTimeFer','sex','age','dist','avisTrainer','nbArrivé','lastPerf',
 'meanPerf',
 'medianPerf',
 'modePerf',
 'recordAbs',
 'gain',
 'nbCourseCouple',
 'nbVictoiresCouple',
 'nb2emeCouple',
 'nb3emeCouple',
 'txReussiteCouple',
 'nonPartant']].astype("int64")

scaler = preprocessing.StandardScaler().fit(X)

y = df[["win"]].astype("int64")

In [123]:
print(X.shape, y.shape)

(200, 19) (200, 1)


In [143]:
X[

,fer,firstTimeFer,sex,age,dist,avisTrainer,nbArrivé,lastPerf,meanPerf,medianPerf,modePerf,recordAbs,gain,nbCourseCouple,nbVictoiresCouple,nb2emeCouple,nb3emeCouple,txReussiteCouple,nonPartant
0,3,0,1,6,2400,1,11,5,4,5,1,719,78820,7,4,1,0,71,0
1,3,0,1,6,2400,2,12,5,5,5,5,728,79970,1,0,0,0,0,0
2,3,0,1,6,2400,1,12,3,4,4,3,737,77690,5,2,1,1,80,0
3,6,0,1,6,2400,2,12,3,4,4,0,725,78110,3,0,0,0,0,0
4,0,0,1,6,2400,2,12,9,4,4,2,729,79860,4,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1985,3,0,0,5,2700,2,12,4,4,3,3,712,105720,5,0,0,2,40,0
1986,3,0,1,5,2700,2,11,0,4,4,0,720,105830,2,0,0,0,0,0
1987,3,0,1,5,2700,2,11,9,4,2,1,733,106320,4,2,0,0,50,0
1988,0,0,1,5,2700,3,11,0,5,5,0,722,107280,1,0,0,0,0,0


In [130]:
scaler = preprocessing.StandardScaler().fit(X)

In [76]:
scaler.transform(X[-16:])

array([[-0.97996827, -0.20892772,  0.33222574, -0.05980558,  0.21824127,
         2.37601827,  0.42285933, -1.09844009, -0.19734967, -0.83638939,
        -0.73478855, -0.28743235,  0.27404864,  1.59279573,  1.57010624,
         1.94126998,  0.75723553,  0.4769169 , -0.1047997 ],
       [ 0.61518786, -0.20892772,  0.33222574, -0.05980558,  0.21824127,
        -0.007769  ,  0.76977734,  0.52636162, -0.19734967,  0.31485761,
         1.1117112 , -1.40797057,  0.32509605,  0.01530122,  0.53776139,
        -0.55794543,  0.75723553,  0.444908  , -0.1047997 ],
       [ 0.08346915, -0.20892772,  0.33222574, -0.05980558,  0.21824127,
        -2.39155626,  0.07594131, -1.09844009, -0.89874108, -0.83638939,
        -0.73478855, -0.78046917,  0.32745208, -0.66076786, -0.49458347,
        -0.55794543, -0.58450712, -0.83544783, -0.1047997 ],
       [ 0.61518786, -0.20892772, -3.01000154, -0.05980558,  0.21824127,
        -2.39155626,  0.42285933,  1.1762823 , -0.89874108, -0.83638939,
        -0.734

### Conditionnal Logit

In [145]:
model = ConditionalLogit(endog=y, exog=scaler.transform(X), groups=df["id"])
model.fit(method="lbfgs", maxiter=1000, full_output=True)

C:\Users\raves\anaconda3\lib\site-packages\statsmodels\discrete\conditional_models.py:80: UserWarning: Dropped 6 groups and 42 observations for having no within-group variance
  warnings.warn(msg)


In [144]:
model.score(X.iloc[0])

C:\Users\raves\anaconda3\lib\site-packages\statsmodels\discrete\conditional_models.py:296: RuntimeWarning: overflow encountered in exp
  exb = np.exp(np.dot(ex, params) + ofs)
C:\Users\raves\anaconda3\lib\site-packages\statsmodels\discrete\conditional_models.py:319: RuntimeWarning: invalid value encountered in double_scalars
  u, v = a + c * h, b + d + e * h
C:\Users\raves\anaconda3\lib\site-packages\statsmodels\discrete\conditional_models.py:348: RuntimeWarning: invalid value encountered in true_divide
  return self._xy[grp] - h / d


array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])

In [78]:
pip install pylogit

Note: you may need to restart the kernel to use updated packages.


In [302]:
pipe = make_pipeline(preprocessing.StandardScaler(), LogisticRegression())
pipe.fit(X[:1500],y[:1500])
#clf = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000).fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [280]:
print(X[2], y[2])

[    3     0     1     6  2400     1    12     3     4     4     3   737
 77690     5     2     1     1    80     0] 1


In [301]:
pipe.predict_proba([X[9]])

array([[0.23910339, 0.26968631, 0.38287318, 0.05802779, 0.05030933]])

In [293]:
pipe.classes_

array([0, 1, 2, 3, 4], dtype=int64)

In [304]:
res2.tail(10)

,classement,nom,id,date,url,fer,firstTimeFer,sex,age,dist,...,medianPerf,modePerf,recordAbs,gain,nbCourseCouple,nbVictoiresCouple,nb2emeCouple,nb3emeCouple,txReussiteCouple,nonPartant
6,0,HAPPY DJOB,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/happ...,3,False,0,5,2700,...,3.0,2,719.0,100730,15,1,3,2,40,False
7,0,HYPER FAST,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hype...,3,False,0,5,2700,...,7.0,7,711.0,102830,5,0,1,2,60,False
8,0,HOOPER DES CHASSES,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hoop...,0,False,1,5,2700,...,3.0,1,727.0,102900,1,0,0,0,0,False
9,2,HERMES LUDOIS,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/herm...,3,False,1,5,2700,...,2.0,2,730.0,102950,1,0,0,1,100,False
10,1,HOLERO LOVE,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hole...,3,False,1,5,2700,...,2.0,1,723.0,105550,4,2,1,0,75,False
11,0,HELENCIO,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hele...,3,False,0,5,2700,...,3.5,3,712.0,105720,5,0,0,2,40,False
12,0,HIATUS,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/hiat...,3,False,1,5,2700,...,4.0,0,720.0,105830,2,0,0,0,0,False
13,4,HARRY CARISAIE,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/harr...,3,False,1,5,2700,...,2.0,1,733.0,106320,4,2,0,0,50,False
14,0,HONNEUR DU CEBE,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/honn...,0,False,1,5,2700,...,5.0,0,722.0,107280,1,0,0,0,0,False
15,0,HAUTAIN,2022110375008,2022-11-03,https://www.letrot.com/stats/fiche-cheval/haut...,6,False,0,5,2700,...,4.0,0,720.0,108530,4,1,0,1,50,False


In [303]:
pipe.score(X[1500:],y[1500:])

0.7272727272727273

In [312]:
pipe.predict_proba(X[-10:])

array([[0.41251139, 0.28134538, 0.02396205, 0.2368595 , 0.04532168],
       [0.15164584, 0.11608796, 0.68116384, 0.04344263, 0.00765972],
       [0.77943041, 0.00294345, 0.00926578, 0.0810417 , 0.12731866],
       [0.1712688 , 0.11118851, 0.69500501, 0.00852741, 0.01401026],
       [0.76005431, 0.07956048, 0.01803439, 0.07017047, 0.07218035],
       [0.45164289, 0.00505256, 0.44704583, 0.06701354, 0.02924518],
       [0.80735886, 0.00339112, 0.01266764, 0.08340467, 0.09317772],
       [0.83886621, 0.0050798 , 0.01080867, 0.05869726, 0.08654806],
       [0.87437219, 0.00172245, 0.00468481, 0.06284231, 0.05637824],
       [0.72260483, 0.00919496, 0.15577519, 0.08033296, 0.03209206]])